In [ ]:
import qlib
qlib.init_qlib(provider_uri='~/.qlib/qlib_data/gta')

In [ ]:
from qlib.data import D
# 引入Timestamp
from pandas import Timestamp

D.calendar(start_time='2010-01-01', end_time='2017-12-31', freq='day') # 2010-01-01是周五，所以返回的是周一和周二
# [Timestamp('2010-01-04 00:00:00'), Timestamp('2010-01-05 00:00:00')]

In [ ]:
D.instruments(market='all')

In [ ]:
instruments = D.instruments(market='sh600809')
D.list_instruments(instruments=instruments, start_time='2010-01-01', end_time='2017-12-31', as_list=True)[:6]

In [ ]:
from qlib.data import D
from qlib.data.filter import NameDFilter
nameDFilter = NameDFilter(name_rule_re='SH[0-9]{4}55') # 匹配以SH开头，后面跟4个数字，最后以55结尾的股票
instruments = D.instruments(market='csi300', filter_pipe=[nameDFilter])
D.list_instruments(instruments=instruments, start_time='2015-01-01', end_time='2016-02-15', as_list=True) # 2015-01-01是周四，所以返回的是周五和周一

In [ ]:
instruments

In [ ]:
from qlib.data import D
from qlib.data.filter import ExpressionDFilter
expressionDFilter = ExpressionDFilter(rule_expression='$close>2000')
instruments = D.instruments(market='csi300', filter_pipe=[expressionDFilter])
D.list_instruments(instruments=instruments, start_time='2015-01-01', end_time='2016-02-15', as_list=True)

In [ ]:
D

In [ ]:
from qlib.data import D
instruments = ['SH600809']
fields = ['$close','$open','$high', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
D.features(instruments, fields, start_time='2018-01-01', end_time='2020-12-31', freq='day').head()

In [ ]:
from qlib.data import D
from qlib.data.filter import NameDFilter, ExpressionDFilter
nameDFilter = NameDFilter(name_rule_re='SH[0-9]{4}55')
expressionDFilter = ExpressionDFilter(rule_expression='$close>Ref($close,1)')
instruments = D.instruments(market='csi300', filter_pipe=[nameDFilter, expressionDFilter])
# instruments = ['SH600809']
fields = ['$close', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
# D.features(instruments, fields, start_time='2010-01-01', end_time='2017-12-31', freq='day').head()
# 查看D.features的数据量
df = D.features(instruments, fields, start_time='2010-01-01', end_time='2017-12-31', freq='day').shape


In [ ]:
# from qlib.data.dataset import DataHandlerLP
# dh = DataHandlerLP.from_df(df)
# print(dh)
# dh.to_pickle(fname, dump_all=True)

In [ ]:
import qlib
from qlib.contrib.data.handler import Alpha158

data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": "csi300",
}



In [ ]:
if __name__ == "__main__":
    qlib.init_qlib()
    h = Alpha158(**data_handler_config)


In [ ]:
# 描述一下h的数据结构
h.get_cols()

In [ ]:
h.fetch()

In [ ]:
h.fetch(col_set='label')

In [ ]:
h.get_backend()

In [ ]:
h.get_feature_config()

In [ ]:
import qlib
qlib.init_qlib(provider_uri='~/.qlib/qlib_data/cn_data')

In [ ]:
from qlib.data.dataset.loader import QlibDataLoader
MACD_EXP = '(EMA($close, 12) - EMA($close, 26))/$close - EMA((EMA($close, 12) - EMA($close, 26))/$close, 9)/$close'
fields = [MACD_EXP] # MACD
names = ['MACD']
labels = ['Ref($close, -2)/Ref($close, -1) - 1'] # label
label_names = ['LABEL']
data_loader_config = {"feature": (fields, names),"label": (labels, label_names)}
data_loader = QlibDataLoader(config=data_loader_config)
df = data_loader.load(instruments='csi300', start_time='2010-01-01', end_time='2017-12-31')
print(df)

In [ ]:
from qlib.data.dataset import DatasetH
# 沪深300股票池代码，在instruments文件夹下必须有对应的文件
# Data Loader
market = 'csi300'
# sh000300.txt
# EMA($close, 10)表示计算close的10日指数加权均线
close_ma = ['EMA($close, 10)', 'EMA($close, 30)'] 
# 下一日收益率, Ref($close, -1)表示下一日收盘价
ma_names = ['EMA10', 'EMA30']
ret = ["Ref($close, -1)/$close-1"] 
ret_name = ['next_ret']
qdl_ma_gp = QlibDataLoader(config={'feature':(close_ma, ma_names), 'label': (ret, ret_name)}) 
# 实例化Data Handler
shared_processors = [DropnaProcessor()]
learn_processors = [CSZScoreNorm()]
infer_processors = [ZScoreNorm(fit_start_time='20190101', fit_end_time='20211231')]
dh_pr_test = DataHandlerLP(instruments='sh000300', 
                           start_time='20190101', 
                           end_time='20211231',
                           process_type=DataHandlerLP.PTYPE_I, 
                           learn_processors=learn_processors,
                           shared_processors=shared_processors,
                           infer_processors=infer_processors,
                           data_loader=qdl_ma_gp)
ds = DatasetH(dh_pr_test, segments={"train": ('20190101', '20201231'), "test": ('20210101', '20211231')})
# 准备训练数据
ds.prepare('train')
# 准备测试数据
ds.prepare('test')

In [ ]:
import qlib
from qlib.data.dataset.loader import QlibDataLoader
qlib.init_qlib(provider_uri='~/.qlib/qlib_data/cn_data')
# 加载原始特征，比如收盘价、最高价
config=(['$close', '$high','EMA($close, 10)', 'EMA($close, 30)'],['close', 'high','EMA10', 'EMA30'])
qdl = QlibDataLoader(config=config)
qdl.load(instruments=['SH600519'], start_time='20190101', end_time='20191231')
# 可以通过freq参数设置周期，默认freq='day'

In [ ]:
# 这次我们加载沪深300成分股的10日和30日收盘价指数加权均价
market = 'csi300' # 沪深300股票池代码，在instruments文件夹下有对应的sh000300.txt
close_ma = ['EMA($close, 10)', 'EMA($close, 30)'] # EMA($close, 10)表示计算close的10日指数加权均线
ma_names = ['EMA10', 'EMA30']
qdl_ma = QlibDataLoader(config=(close_ma, ma_names))
qdl_ma.load(instruments=market, start_time='20150101', end_time='20210110')

In [ ]:
# 沪深300股票池代码，在instruments文件夹下有对应的sh000300.txt
market = 'csi300' 
# EMA($close, 10)表示计算close的10日指数加权均线
close_ma = ['EMA($close, 10)', 'EMA($close, 30)']
ma_names = ['EMA10', 'EMA30']
ret = ["Ref($close, -1)/$close-1"] 
# 下一日收益率, Ref($close, -1)表示下一日收盘价
ret_name = ['next_ret']
qdl_ma_gp = QlibDataLoader(config={'feature':(close_ma, ma_names), 'label': (ret, ret_name)}) 
qdl_ma_gp.load(instruments=market, start_time='20150101', end_time='20210110')

In [ ]:
from qlib.data.dataset.handler import DataHandlerLP
from qlib.data.dataset.processor import CSZScoreNorm, DropnaProcessor

qdl = QlibDataLoader(config=(['$close/Ref($close, 1)-1'],['Return']))
df = qdl.load(instruments='csi300', start_time='20210101', end_time='20210430') 
# 是否有空值
df.isna().sum()
# 原始数据分布
# df.xs('2021-01-05').hist()
# 实例化DataHandler
dh = DataHandlerLP(instruments='csi300', start_time='20150101', end_time='20210120',learn_processors=[DropnaProcessor(), CSZScoreNorm()],data_loader=qdl)
df_hdl = dh.fetch(data_key=DataHandlerLP.DK_L) # 获取处理后的数据，处理过程为先去空值，再截面标准化。
# 查看是否还存在空值
df_hdl.isna().sum()
# CSZScoreNorm截面标准化处理后的数据分布
# df_hdl.xs('2021-01-05').hist()